In [1]:
import os
os.chdir('..')

In [2]:
from math import pi
from sim_circuit import *
from util import print_state_table, is_close

## Exercise 1

The famous mathematician John von Neumann's proposed a solution for getting fair results by tossing a biased coin. The solution is succinctly described like this:

* Toss the coin twice.
* If the results match (HH or TT), start over, forgetting both results.
* If the results differ (HT or TH), use the first result, forgetting the second.

Which Bell state offers a quantum solution to von Neumann's problem?

**Answer:**

The second two Bell states.

When we measure one of the last two Bell states, we will get '01' or '10'.

## Exercise 2

Implement circuits that encode the remaining two Bell states.

**Answer**: The second Bell state

In [3]:
q = QuantumRegister(2)
qc = QuantumCircuit(q)

qc.h(q[0])
qc.cx(q[0], q[1])
qc.z(q[1])

bell_state2 = qc.run()

In [4]:
bell_state2

[0.7071067811865475, 0.0, 0.0, -0.7071067811865475]

In [5]:
print_state_table(bell_state2)



Outcome   Binary        Amplitude        Magnitude   Direction      Amplitude Bar         Probability
----------------------------------------------------------------------------------------------------
  0         00     +0.7071 + 0.0000*i     0.7071     0.0           ##############           0.5
  1         01     +0.0000 + 0.0000*i     0.0000                                            0.0
  2         10     +0.0000 + 0.0000*i     0.0000                                            0.0
  3         11     -0.7071 + 0.0000*i     0.7071     180.0         ##############           0.5


**Answer**: The fourth Bell state

In [6]:
q = QuantumRegister(2)
qc = QuantumCircuit(q)

qc.h(q[0])
qc.x(q[1])
qc.cx(q[0], q[1])
qc.z(q[1])

bell_state4 = qc.run()

In [7]:
bell_state4

[0.0, 0.7071067811865475, -0.7071067811865475, 0.0]

In [8]:
print_state_table(bell_state4)



Outcome   Binary        Amplitude        Magnitude   Direction      Amplitude Bar         Probability
----------------------------------------------------------------------------------------------------
  0         00     +0.0000 + 0.0000*i     0.0000                                            0.0
  1         01     +0.7071 + 0.0000*i     0.7071     0.0           ##############           0.5
  2         10     -0.7071 + 0.0000*i     0.7071     180.0         ##############           0.5
  3         11     +0.0000 + 0.0000*i     0.0000                                            0.0


## Exercise 3: Implementing the geometric distribution

The geometric distribution models the number of times a process must be repeated before a successful outcome is achieved. If the probability of success is $p$, then the probability of having $k$ failures before the first success is:
$(1-p)^{k}p$.

Verify that the following circuit:

```
q = QuantumRegister(n)
qc = QuantumCircuit(q)

for i in range(len(q)):
    qc.ry(theta, q[i])

for i in range(len(q) - 1):
    qc.cry(pi - theta, q[i], q[i+1])

z = qc.run()
```

prepares a quantum state $z$ with the following properties:

1. The amplitudes in the state $z$ at indices $2^n - 2^k$ for $0 \le k < n$ (which start with $k$ digits of 1, and end with $n-k$ digits of 0) are $z_{2^n - 2^k} = cos^{k}\frac{\theta}{2} \sin\frac{\theta}{2}$, and the probabilities: $|z_{2^n - 2^k}|^2 = cos^{2k}\frac{\theta}{2} \sin^2\frac{\theta}{2} = (1-p)^{k}p$, with $p = \sin^2\frac{\theta}{2}$, matching the geometric distribution probabilities.

2. Amplitude $z_0$ is the "left over" amplitude accounting for the infinite tail of the geometric distribution.

3. All other amplitudes are 0.

4. The table state representation for $n = 3$ and $\theta = 0.8\pi$ matches.

**Answer:**

In [9]:
# prepare state with n = 3 and theta = 0.8pi
n = 3
theta = 0.8*pi

q = QuantumRegister(n)
qc = QuantumCircuit(q)

for i in range(len(q)):
    qc.ry(theta, q[i])

for i in range(len(q) - 1):
    qc.cry(pi - theta, q[i], q[i+1])

z = qc.run()

In [10]:
n  = 3
special_indices = [2**n-2**k for k in range(n)]

k = n-1
for i in range(1, 2**n):
    if i in special_indices:
        assert is_close(z[i], cos(theta/2)**k*sin(theta/2))
        k = k - 1
    else:
        assert(abs(z[i]) < 0.0001)

In [11]:
assert is_close(abs(z[0])**2, 1 - sum([abs(z[i])**2 for i in range(1, 2**n)]))

In [12]:
print_state_table(z)



Outcome   Binary        Amplitude        Magnitude   Direction      Amplitude Bar         Probability
----------------------------------------------------------------------------------------------------
  0        000     +0.0295 + 0.0000*i     0.0295     0.0                                    0.0009
  1        001     +0.0000 + 0.0000*i     0.0000                                            0.0
  2        010     +0.0000 + 0.0000*i     0.0000                                            0.0
  3        011     +0.0000 + 0.0000*i     0.0000                                            0.0
  4        100     +0.0908 + 0.0000*i     0.0908     0.0           #                        0.0082
  5        101     +0.0000 + 0.0000*i     0.0000                                            0.0
  6        110     +0.2939 + 0.0000*i     0.2939     0.0           #####                    0.0864
  7        111     +0.9511 + 0.0000*i     0.9511     0.0           ###################      0.9046
